# Fed-BioMed Researcher to train a federated scikit learn model.

## Purpose of the exercise :

Three datasets `perp-c1.csv` , `perp-c2.csv` and `perp-c3.csv` have been generated with a target column of 3 different classes.
We will fit a Perceptron (classifier) using Federated Learning.

## Extending this notebook to any incremental learning scikit model:

The same federated learning scheme below applies to any sklearn model supporting the method partial_fit():

A family of models could be naturally imported in Fed-BioMed, following the same approach. For example: 
- Naive Bayes.  
- Logistic regression,
- SVM/SVC (linear and non-linear), 
- perceptron, 
- KMeans, 
- incremental PCA, 
- mini batch dictionary learning, 
- latent Dirichlet annotation, 

## Get the data 

We use the make_classification dataset from sklearn datasets

In [ ]:
from sklearn import datasets
import numpy as np

In [ ]:
X,y = datasets.make_classification(n_samples=300, n_features=20,n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0,shuffle=True, random_state=123)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
C1 = X[:150,:]
C2 = X[150:250,:]
C3 = X[250:300,:]

In [ ]:
y1 = y[:150].reshape([150,1])
y2 = y[150:250].reshape([100,1])
y3 = y[250:300].reshape([50,1])

In [ ]:
C1.shape ,C2.shape , C3.shape , y1.shape, y2.shape, y3.shape

In [ ]:
C2.shape

In [ ]:
n1 = np.concatenate((C1, y1), axis=1)
np.savetxt('./data/perp-c1.csv',n1,delimiter=',')

In [ ]:
n2 = np.concatenate((C2, y2), axis=1)
np.savetxt('./data/perp-c2.csv',n2,delimiter=',')

In [ ]:
n3 = np.concatenate((C3, y3), axis=1)
np.savetxt('./data/perp-c3.csv',n3,delimiter=',')

## Setting the node up
Before running this notebook you need to configure 2 nodes: <br/>
* **Node 1 :** `fedbiomed node --path node-1 dataset add`
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset (you can write 'perp' always and it will be good)
  * Pick the perp-c1.csv file in your machine.
  * Check that your data has been added in node 1 by executing `fedbiomed node --path node-1 dataset list`
  * Run the node using `fedbiomed node --path node-1 start`. <br/>

* **Node 2 :** Open a second terminal and run `fedbiomed node -d node-2 dataset add` 
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset (you can write 'perp' always and it will be good)
  * Pick the perp-c2.csv file in your machine.
  * Check that your data has been added in node 2 by executing `fedbiomed node --path ./node-2 dataset list`
  * Run the node using `fedbiomed node --path node-2 start`.
 

 Wait until you get `Starting task manager`, it means node is online.


**model_args** is a dictionnary containing your model arguments, in case of SGDRegressor this will be max_iter and tol.

**training_args** is a dictionnary with parameters , related to Federated Learning. 

In [ ]:
from fedbiomed.common.metrics import MetricTypes

n_features = 20
n_classes = 2

model_args = { 'max_iter':1000, 'tol': 1e-3 , 
               'n_features' : n_features, 'n_classes' : n_classes}

training_args = {   
    'epochs': 5,
    'loader_args': { 'batch_size': 1, },
    'test_ratio': 0.2,
    'test_metric': MetricTypes.ACCURACY,
    'test_on_local_updates': True,
    'test_on_global_updates': True,
    'test_batch_size': 32,
}

Hereafter the template of the class you should provide to Fedbiomed :
    
**training_data** : you must return here the (X,y) that must be of the same type of 
your method partial_fit parameters.

In [ ]:
from fedbiomed.common.training_plans import FedPerceptron
from fedbiomed.common.data import DataManager
class PerceptronTrainingPlan(FedPerceptron):
    def training_data(self):
        NUMBER_COLS = 20
        dataset = pd.read_csv(self.dataset_path,header=None,delimiter=',')
        X = dataset.iloc[:,0:NUMBER_COLS].values
        y = dataset.iloc[:,NUMBER_COLS]
        return DataManager(dataset=X,target=y.values,  shuffle=True)

In [ ]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['perp']
rounds = 5

# search for corresponding datasets across nodes datasets
exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=PerceptronTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)


In [ ]:
# train experiments
exp.run()

Save trained model to file

In [ ]:
exp.training_plan().export_model('./trained_model')

## Lets validate the trained model with the test dataset perp-c3.csv.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('./data/perp-c3.csv')

In [ ]:
X_test = data.iloc[:,:n_features]
y_test = data.iloc[:,n_features]

F1 score computed with federated algorithm :

For that, we are exporting `exp.aggregated_params()` containing models parameters collected at the end of each round

In [ ]:
from sklearn.metrics import f1_score
loss_metric = f1_score
    
testing_error = []

for i in range(rounds):
    fed_model = exp.training_plan().model()
    fed_model.coef_ = exp.aggregated_params()[i]['params']['coef_']
    fed_model.intercept_ = exp.aggregated_params()[i]['params']['intercept_']
    metric = loss_metric(fed_model.predict(X_test),y_test.ravel())
    print('F1 score metric: ', metric, )
    testing_error.append(metric)